In [21]:
#데이터 구조를 위하여
import pandas as pd
import numpy as np

#웹 크롤링을 위하여
from bs4 import BeautifulSoup
from selenium import webdriver

# 크롤링 통신하기 위해서
import requests
import json
from pandas.io.json import json_normalize

#webdriver의 select와 key를 위하여
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import time

In [22]:
driver = webdriver.Chrome('chromedriver')

In [23]:
#버튼이 동작하게 해보자
def click(area):
    driver.get("https://www.starbucks.co.kr/store/store_map.do?disp=locale")
    time.sleep(1)
    driver.implicitly_wait(10)
    
    #각 지정 지역 클릭
    ss=driver.find_element_by_class_name('sido_arae_box')
    s1=ss.find_elements_by_tag_name('li')
    time.sleep(1)
    driver.implicitly_wait(10)
    s1[area].click()
    
    #로딩 대기
    time.sleep(1)
    driver.implicitly_wait(10)
    
    #전체 클릭
    gg= driver.find_element_by_class_name('gugun_arae_box')
    g1=gg.find_elements_by_tag_name('li')
    
    time.sleep(1)
    driver.implicitly_wait(10)
    g1[0].click()
    
    #로딩 대기
    time.sleep(1)
    driver.implicitly_wait(10)
    
    #소스코드
    source=driver.page_source
    bs= BeautifulSoup(source,'lxml')
    return bs
    

In [30]:
def getinfo(bs):
    #매장정보 접근
    ul = bs.find('ul',class_='quickSearchResultBoxSidoGugun')
    sb_list =ul.find_all('li')
    
    time.sleep(1)
    driver.implicitly_wait(10)
    
    #매장의 정보를 탐을 리스트 생성
    name=[]
    lat=[]
    long=[]
    add=[]
    
    #각 매장의 정보를 받아온다
    for sb in sb_list:
        name.append(sb['data-name'])
        lat.append(sb['data-lat'])
        long.append(sb['data-long'])
        add.append(sb.find('p').text)
    
    #데이터프레임 생성    
    area=pd.DataFrame([name,lat,long,add], index=['매장이름','위도','경도','주소']).T
    return area

In [31]:
#실행 !!
#서울=0 경기=1

bs = click(0)
ss_sb = getinfo(bs)
bs=click(1)
gg_sb =getinfo(bs)

<ipython-input-23-f93515c01a96>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ss=driver.find_element_by_class_name('sido_arae_box')
C:\Users\KI\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:358: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
<ipython-input-23-f93515c01a96>:19: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  gg= driver.find_element_by_class_name('gugun_arae_box')


In [28]:
len(ss_sb),len(gg_sb)

(567, 362)

In [29]:
gg_sb.head(10)

,매장이름,위도,경도,주소
0,남이섬,37.806624,127.52527,경기도 가평군 가평읍 북한강변로 1054 북한강변로 10541522-3232
1,소노캄고양,37.6618,126.7507,경기도 고양시 일산동구 장항동 1755 1522-3232
2,일산애니골DT,37.6748,126.793181,경기도 고양시 일산동구 무궁화로 294 (풍동)1522-3232
3,동국대일산병원,37.676411,126.806405,경기도 고양시 일산동구 식사동 814 경기도 고양시 일산동구 동국로 271522-3232
4,일산풍동DT,37.662091,126.800900,경기도 고양시 일산동구 백마로 475 (풍동)1522-3232
5,백석중앙로,37.641100,126.790987,경기도 고양시 일산동구 강송로 33 (백석동)1522-3232
6,일산식사,37.6779381,126.812161,경기도 고양시 일산동구 위시티2로11번길 31 (식사동)1522-3232
7,풍산이마트,37.673769,126.786883,경기도 고양시 일산동구 무궁화로 237 (중산동)1522-3232
8,백석역DT,37.6439377,126.7873856,"경기도 고양시 일산동구 중앙로 1054, 1층 (백석동)1522-3232"
9,백석,37.643888,126.786153,경기도 고양시 일산동구 중앙로 1059 (백석동)1522-3232


In [32]:
gg_sb.head(10)

,0,1,2,3,4,5,6,7,8,9,...,352,353,354,355,356,357,358,359,360,361
매장이름,남이섬,소노캄고양,일산애니골DT,동국대일산병원,일산풍동DT,백석중앙로,일산식사,풍산이마트,백석역DT,백석,...,파주금촌역,파주금릉역,파주운정,파주P.아울렛3F,파주문산DT,파주산내,파주야당DT,파주운정이마트,지행역,동두천DT
위도,37.806624,37.6618,37.6748,37.676411,37.662091,37.641100,37.6779381,37.673769,37.6439377,37.643888,...,37.764081,37.752207,37.71203,37.768974,37.8464380469309,37.728707,37.70346,37.710647,37.892653,37.888925
경도,127.52527,126.7507,126.793181,126.806405,126.800900,126.790987,126.812161,126.786883,126.7873856,126.786153,...,126.774205,126.765613,126.744499,126.696320,126.789040084356,126.735514,126.75964,126.74545,127.053463,127.057066
주소,경기도 가평군 가평읍 북한강변로 1054 북한강변로 10541522-3232,경기도 고양시 일산동구 장항동 1755 1522-3232,경기도 고양시 일산동구 무궁화로 294 (풍동)1522-3232,경기도 고양시 일산동구 식사동 814 경기도 고양시 일산동구 동국로 271522-3232,경기도 고양시 일산동구 백마로 475 (풍동)1522-3232,경기도 고양시 일산동구 강송로 33 (백석동)1522-3232,경기도 고양시 일산동구 위시티2로11번길 31 (식사동)1522-3232,경기도 고양시 일산동구 무궁화로 237 (중산동)1522-3232,"경기도 고양시 일산동구 중앙로 1054, 1층 (백석동)1522-3232",경기도 고양시 일산동구 중앙로 1059 (백석동)1522-3232,...,경기도 파주시 문화로 109 (금촌동)1522-3232,"경기도 파주시 금릉역로 84, 청원센트럴타워 1층 (금촌동)1522-3232",경기도 파주시 와석순환로 1351522-3232,경기도 파주시 탄현면 필승로 200 1312호1522-3232,경기도 파주시 파주읍 통일로 1555 (봉서리)1522-3232,경기도 파주시 청암로17번길 29 (목동동)1522-3232,경기도 파주시 경의로 966 (야당동)1522-3232,경기도 파주시 한울로 123 (동패동)1522-3232,경기도 동두천시 지행로 55 (지행동)1522-3232,경기도 동두천시 평화로 2254 (지행동)1522-3232
